In [21]:
from sentence_transformers import SentenceTransformer
import numpy as np
import torch
import pickle
from sentence_transformers.util import semantic_search
import json 

In [22]:
model_path = "data/model/st-model/16500"
# model_path = "data/model/st-model-1/37500"
# retriever_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
retriever_model = SentenceTransformer(model_path)

In [23]:
import pandas as pd
df = pd.read_excel("data/final_data.xlsx")

In [24]:
ques_df = df[['Q1','Q2','Q3','Q4','Mujib Relevant']]


In [25]:
mujib_relevant_ques = ques_df[ques_df['Mujib Relevant'] == "YES"]


In [26]:
q1 = mujib_relevant_ques['Q1'].dropna()
q2 = mujib_relevant_ques['Q2'].dropna()
q3 = mujib_relevant_ques['Q3'].dropna()
q4 = mujib_relevant_ques['Q4'].dropna()

In [27]:
print(f"Total Q1 : {len(q1)}")
print(f"Total Q2 : {len(q2)}")
print(f"Total Q3 : {len(q3)}")
print(f"Total Q4 : {len(q4)}")

Total Q1 : 265
Total Q2 : 244
Total Q3 : 127
Total Q4 : 35


In [28]:
q1_ques= []

for q in q1:
    q1_ques.append(q)

q1_idx = q1.index.tolist()

len(q1_ques) , len(q1_idx)

(265, 265)

In [29]:
q2_ques= []

for q in q2:
    q2_ques.append(q)

q2_idx = q2.index.tolist()

len(q2_ques) , len(q2_idx)

(244, 244)

In [30]:
q3_ques= []

for q in q3:
    q3_ques.append(q)

q3_idx = q3.index.tolist()

len(q3_ques) , len(q3_idx)

(127, 127)

In [31]:
q4_ques= []

for q in q4:
    q4_ques.append(q)

q4_idx = q4.index.tolist()

len(q4_ques) , len(q4_idx)

(35, 35)

In [32]:
q1_embds = retriever_model.encode(q1_ques)

In [33]:

def evaluate(qN):
    
    if qN == 2:
        ques = q2_ques
        index = q2_idx
    elif qN == 3:
        ques = q3_ques
        index = q3_idx
    elif qN == 4:
        ques = q4_ques
        index = q4_idx
    else:
        return "Invalid question number"
            
        
    model_idx = []
    for q in ques:
        q_emb = retriever_model.encode(q)
        query_embeddings = torch.FloatTensor(q_emb)
        hits = semantic_search(query_embeddings, q1_embds, top_k=1)

        # print(hits[0][0]['corpus_id'])
        model_idx.append(hits[0][0]['corpus_id'])

    f_d = []
    for i in range(len(model_idx)):
        f = model_idx[i]
        # if f <= q2_idx[-1]:
        f_d.append(q1_idx[model_idx[i]])

    count = 0
    for i in range(len(f_d)):
        if f_d[i] == index[i]:
            count += 1

    print(f"Successfully indexed: {count} questions out of {len(index)}")

    model_accuracy = count/len(index)
    print("Models Accuracy : ", model_accuracy)

In [34]:
evaluate(2)

Successfully indexed: 177 questions out of 244
Models Accuracy :  0.7254098360655737


In [35]:
evaluate(3)

Successfully indexed: 90 questions out of 127
Models Accuracy :  0.7086614173228346


In [36]:
evaluate(4)

Successfully indexed: 26 questions out of 35
Models Accuracy :  0.7428571428571429


In [37]:
def generate_report():
    
    print('='*15+" Testing with Q2 "+'='*15)
    print()
    evaluate(2)
    print()
    print('='*15+" Testing with Q3 "+'='*15)
    print()
    evaluate(3)
    print()
    print('='*15+" Testing with Q4 "+'='*15)
    print()
    evaluate(4)

In [38]:
generate_report()

=============== Testing with Q2 ===============

Successfully indexed: 177 questions out of 244
Models Accuracy :  0.7254098360655737

=============== Testing with Q3 ===============

Successfully indexed: 90 questions out of 127
Models Accuracy :  0.7086614173228346

=============== Testing with Q4 ===============

Successfully indexed: 26 questions out of 35
Models Accuracy :  0.7428571428571429
